In [1]:
import cv2
from src.data.image_loaders import VehiculesSquareImageLoader, VehiculesImageLoader
from src.data.dataset_loaders import VehiculesDatasetLoader
import matplotlib.pyplot as plt
import os
import shutil 

In [12]:
annotation_filepath = '../../data/raw/vehicules/Annotations1024/'
image_file_path = '../../data/raw/vehicules/Vehicules1024'

['00000000.txt',
 '00000001.txt',
 '00000002.txt',
 '00000003.txt',
 '00000004.txt',
 '00000006.txt',
 '00000007.txt',
 '00000008.txt',
 '00000009.txt',
 '00000010.txt',
 '00000011.txt',
 '00000012.txt',
 '00000013.txt',
 '00000014.txt',
 '00000015.txt',
 '00000016.txt',
 '00000017.txt',
 '00000018.txt',
 '00000019.txt',
 '00000020.txt',
 '00000021.txt',
 '00000022.txt',
 '00000025.txt',
 '00000026.txt',
 '00000027.txt',
 '00000029.txt',
 '00000030.txt',
 '00000031.txt',
 '00000032.txt',
 '00000033.txt',
 '00000035.txt',
 '00000036.txt',
 '00000037.txt',
 '00000038.txt',
 '00000040.txt',
 '00000041.txt',
 '00000042.txt',
 '00000043.txt',
 '00000044.txt',
 '00000045.txt',
 '00000046.txt',
 '00000047.txt',
 '00000048.txt',
 '00000049.txt',
 '00000050.txt',
 '00000051.txt',
 '00000052.txt',
 '00000053.txt',
 '00000054.txt',
 '00000055.txt',
 '00000056.txt',
 '00000057.txt',
 '00000058.txt',
 '00000059.txt',
 '00000060.txt',
 '00000061.txt',
 '00000062.txt',
 '00000063.txt',
 '00000064.txt

In [60]:
images = os.listdir(image_file_path)
images = [img.split("_")[0] for img in images if img.split("_")[1] == 'co.png']

Check for images with annotations and copy both the image and annotation file into cascade

In [ ]:
files_with_pair = []
for file_name in os.listdir(annotation_filepath):
    file_num = file_name.split(".")[0]
    if file_num in images:
        files_with_pair.append(file_num)

imgs_with_pair = ['_'.join([x, 'co.png']) for x in files_with_pair]
imgs_with_pair = ['/'.join([image_file_path, x]) for x in imgs_with_pair]

for file in imgs_with_pair:
    shutil.copy2(file, '../../data/interim/cascades')

ann_with_pairs = ['.'.join([x, 'txt']) for x in files_with_pair]
ann_with_pairs = ['/'.join([annotation_filepath, x]) for x in ann_with_pairs]
for file in ann_with_pairs:
    shutil.copy2(file, '../../data/interim/cascades' )

Load images from cascade folder

In [19]:
BND_BOX_SIZE = (256, 256)
DATASET_PATH = '../../data/interim/cascades/'

In [34]:
dataset_loader = VehiculesDatasetLoader(
    image_loader=VehiculesImageLoader()
)


images = dataset_loader.load_dataset(
    input_folder_filepath=DATASET_PATH,
    images_files_types=('png',),
    annotations_files_types=('txt',),
    verbose=True
)

100%|██████████| 596/596 [00:19<00:00, 30.22it/s]


Create pos.txt file

In [44]:
def read_images(input_images, output_file):
    result = []
    for image in input_images:
        line = []
        input_dir = image.image_dir[19:]
        num_obj = str(len(image.bnd_boxes))
        line.append(input_dir)
        line.append(num_obj)
        for bnd in image.bnd_boxes:
            x = bnd[0]
            y = bnd[1]
            w = bnd[2] - x
            h = bnd[7] - y
           
            if x + w >1024:
                w = 1024 - x
            if y + h > 1024:
                h = 1024 - h
            box = [x,y,w,h]
            box = [str(x) for x in box]
            line.extend(box)
        str_line = ' '.join(line)
        result.append(str_line)
        # print(str_line)
        with open(output_file, "w") as f:
            for line in result:
                f.write(line)
                f.write('\n')
        

In [45]:
read_images(images, "../../data/interim/pos.txt")

Create negative samples 

In [36]:
output_dir = "../../data/interim/negative/"
negative_imgs = []
for id, image in enumerate(images):
    output_path = ''.join([output_dir, 'neg', str(id), '.png'])
    negative_imgs.append(output_path)
    r_image = cv2.cvtColor(image.get_random_box(400, 400), cv2.COLOR_BGR2RGB)

    cv2.imwrite(output_path, r_image)

Create neg.bg file

In [47]:
with open('../../data/interim/neg.txt', 'w') as text_file:
    for filename in os.listdir('../../data/interim/negative'):
        text_file.write('negative/'+ filename + '\n')

Copy positive samples into positive

In [94]:
for file in imgs_with_pair:
    shutil.copy2(file, '../../data/interim/positive')

In [ ]:
cascade = cv2.CascadeClassifier('../../data/interim/cascade_train/cascade.xml')

In [ ]:
file = "../../data/interim/cascades/00000085_co.png"
img = cv2.imread(file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [11]:
rectangles = cascade.detectMultiScale(img)

KeyboardInterrupt: 

TypeError: Expected Ptr<cv::UMat> for argument 'mat'